# Ring Learning With Errors


**Source:**

https://blog.openmined.org/build-an-homomorphic-encryption-scheme-from-scratch-with-python/

## Imports

In [15]:
import numpy as np
from numpy.polynomial import polynomial as poly

## Funciphertextions

In [16]:
# Polynomial multiplication in the ring: Multiplies two polynomials (x and y), then reduces the product modulo the ciphertext modulus and the polynomial modulus.
def polymul(x, y, ciphertext_mod, poly_mod):
    # Multiply x and y, reduce modulo ciphertext_mod, divide by poly_mod, take the remainder, and then reduce modulo ciphertext_mod again
    return np.int64(np.round(poly.polydiv(poly.polymul(x, y) % ciphertext_mod, poly_mod)[1] % ciphertext_mod))

# Polynomial addition in the ring: Adds two polynomials (x and y), then reduces the sum modulo the ciphertext modulus and the polynomial modulus.
def polyadd(x, y, ciphertext_mod, poly_mod):
    # Add x and y, reduce modulo ciphertext_mod, divide by poly_mod, take the remainder, and then reduce modulo ciphertext_mod again
    return np.int64(np.round(poly.polydiv(poly.polyadd(x, y) % ciphertext_mod, poly_mod)[1] % ciphertext_mod))

# Key generation function
def keygen(size, ciphertext_mod, poly_mod):
    # Generate a random binary secret key
    secret_key = np.random.randint(0, 2, size=size, dtype=np.int64)
    # Generate a random polynomial A
    A = np.random.randint(0, ciphertext_mod, size=size, dtype=np.int64)
    # Generate a random error term with a normal distribution
    error = np.int64(np.random.normal(0, 2, size=size))
    # Calculate b as the addition of -A*secret_key and -error in the ring
    b = polyadd(polymul(-A, secret_key, ciphertext_mod, poly_mod), -error, ciphertext_mod, poly_mod)
    # Return public key (A, b) and secret key
    return (A, b), secret_key

# Encryption function
def encrypt(public_key, size, ciphertext_mod, plaintext_mod, poly_mod, plaintext):
    # Encode the plaintext as a polynomial with the plaintext in the first coefficient
    m = np.array([plaintext] + [0] * (size - 1), dtype=np.int64) % plaintext_mod
    # Scale the plaintext polynomial by delta, which is the quotient of the ciphertext modulus and plaintext modulus
    delta = ciphertext_mod // plaintext_mod
    scaled_m = delta * m  % ciphertext_mod
    # Generate random error terms e1 and e2
    e1 = np.int64(np.random.normal(0, 2, size=size))
    e2 = np.int64(np.random.normal(0, 2, size=size))
    # Generate a random binary polynomial u
    u = np.random.randint(0, 2, size, dtype=np.int64)
    # Compute the first part of the ciphertext
    ciphertext0 = polyadd(polymul(public_key[0], u, ciphertext_mod, poly_mod), e2, ciphertext_mod, poly_mod)
    # Compute the second part of the ciphertext
    ciphertext1 = polyadd(polyadd(polymul(public_key[1], u, ciphertext_mod, poly_mod), e1, ciphertext_mod, poly_mod), scaled_m, ciphertext_mod, poly_mod)
    # Return the ciphertext tuple
    return (ciphertext0, ciphertext1)

# Decryption function
def decrypt(secret_key, size, ciphertext_mod, plaintext_mod, poly_mod, ciphertext):
    # Combine the two parts of the ciphertext using the secret key
    scaled_pt = polyadd(polymul(ciphertext[0], secret_key, ciphertext_mod, poly_mod), ciphertext[1], ciphertext_mod, poly_mod)
    # Scale down to the plaintext modulus and round to get the plaintext polynomial
    decrypted_poly = np.round(scaled_pt * plaintext_mod / ciphertext_mod) % plaintext_mod
    # Return the first coefficient of the decrypted polynomial as the plaintext
    return int(decrypted_poly[0])

# Function to multiply a ciphertext with a plaintext integer
def mul_plain(ciphertext, plaintext, ciphertext_mod, plaintext_mod, poly_mod):
    # Determine the size of the polynomial
    size = len(poly_mod) - 1
    # Encode the plaintext as a polynomial
    m = np.array([plaintext] + [0] * (size - 1), dtype=np.int64) % plaintext_mod
    # Multiply the first part of the ciphertext with the plaintext polynomial
    new_ciphertext0 = polymul(ciphertext[0], m, ciphertext_mod, poly_mod)
    # Multiply the second part of the ciphertext with the plaintext polynomial
    new_ciphertext1 = polymul(ciphertext[1], m, ciphertext_mod, poly_mod)
    # Return the new ciphertext tuple
    return (new_ciphertext0, new_ciphertext1)

## Parameters

In [17]:
# Message
message = 5
# Multiplication factor
factor = 6

# polynomial modulus degree
polynomial_degree = 2**4
# ciphertext modulus
ciphertext_mod = 2**15
# plaintext modulus
plaintext_mod = 2**8
# polynomial modulus
poly_mod = np.array([1] + [0] * (polynomial_degree - 1) + [1])

In [18]:
# Key generation
public_key, secret_key = keygen(polynomial_degree, ciphertext_mod, poly_mod)

# Encryption
cipher = encrypt(public_key, polynomial_degree, ciphertext_mod, plaintext_mod, poly_mod, message)

# Multiplication with plaintext integer/factor
cipher_result = mul_plain(cipher, factor, ciphertext_mod, plaintext_mod, poly_mod)

# Decryption
result = decrypt(secret_key, polynomial_degree, ciphertext_mod, plaintext_mod, poly_mod, cipher_result)

In [27]:
print("Message:", message)
print("Factor:", factor, '\n')
print("secret_key:")
print(secret_key, '\n')
print("public_key (A):")
print(public_key[1], '\n')
print("public_key (b):")
print(public_key[0], '\n')
print("cipher:")
print(cipher, '\n')
print("cipher_result:")
print(cipher_result, '\n')
print("result (encryption):", result)
print("result (clear):", message * factor, '\n')

Message: 5
Factor: 6 

secret_key:
[0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 1] 

public_key (A):
[15664 17947 15822 20317  1143 23820 27415  3225 30524 18444 24770 27375
 32759 15991 24215  7181] 

public_key (b):
[22633  2630  5623 15534 22325 10513 16853 12255 10976  8142 23689  7503
 27703  2146 12013 28266] 

cipher:
(array([ 2843, 13383, 25599, 23924,  7048, 32216, 27626, 27111, 28123,
       25161, 27453, 31349, 29410, 24361, 19411,  7688]), array([30318, 19498,  1714, 11355, 25217, 11465, 29071, 13259, 31877,
         856, 17800, 25521,   401, 21334, 11300,  7725])) 

cipher_result:
(array([17058, 14762, 22522, 12472,  9520, 29456,  1916, 31594,  4898,
       19894,   878, 24254, 12620, 15094, 18162, 13360]), array([18068, 18684, 10284,  2594, 20230,  3254, 10586, 14018, 27422,
        5136,  8496, 22054,  2406, 29700,  2264, 13582])) 

result (encryption): 30
result (clear): 30 

